# Notebook 2: PDE-Based Image Restoration

This notebook implements PDE (Partial Differential Equation) based inpainting using the Navier-Stokes approach. The algorithm uses fluid dynamics principles to propagate information from known regions into damaged areas.


In [ ]:
import sys
import os
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy import ndimage

sys.path.append('..')
from utils import load_image, save_image, ensure_dir, get_image_files

print("Libraries imported successfully!")


## 1. PDE-Based Inpainting Implementation

The Navier-Stokes based inpainting algorithm:
1. Uses the Laplacian operator to smooth the image
2. Propagates information from boundaries into the damaged region
3. Iteratively fills the masked area


In [ ]:
def pde_inpaint(image, mask, iterations=100, dt=0.1):
    """
    PDE-based inpainting using Navier-Stokes approach.
    
    Args:
        image: Input damaged image (RGB)
        mask: Binary mask (255 for damaged regions, 0 for known)
        iterations: Number of iterations
        dt: Time step for diffusion
        
    Returns:
        Restored image
    """
    # Convert to float for processing
    img = image.astype(np.float32)
    mask_bool = (mask == 255).astype(np.float32)
    
    # Create mask for known regions (inverse)
    known_mask = 1.0 - mask_bool
    
    # Initialize result with known pixels
    result = img * known_mask[..., np.newaxis]
    
    # Create Laplacian kernel
    laplacian_kernel = np.array([[0, 1, 0],
                                 [1, -4, 1],
                                 [0, 1, 0]], dtype=np.float32)
    
    for i in range(iterations):
        # Compute Laplacian for each channel
        laplacian = np.zeros_like(result)
        for c in range(3):
            laplacian[:, :, c] = cv2.filter2D(result[:, :, c], -1, laplacian_kernel)
        
        # Update only masked regions
        update = dt * laplacian
        result = result + update * mask_bool[..., np.newaxis]
        
        # Keep known regions fixed
        result = result * known_mask[..., np.newaxis] + img * known_mask[..., np.newaxis]
        
        # Clamp values to valid range
        result = np.clip(result, 0, 255)
    
    return result.astype(np.uint8)


def pde_inpaint_improved(image, mask, iterations=200, dt=0.1, edge_preserving=True):
    """
    Improved PDE-based inpainting with edge preservation.
    
    Args:
        image: Input damaged image (RGB)
        mask: Binary mask (255 for damaged regions)
        iterations: Number of iterations
        dt: Time step
        edge_preserving: Use edge-preserving diffusion
        
    Returns:
        Restored image
    """
    img = image.astype(np.float32)
    mask_bool = (mask == 255).astype(np.float32)
    known_mask = 1.0 - mask_bool
    
    result = img * known_mask[..., np.newaxis]
    
    # Convert to grayscale for edge detection
    gray = cv2.cvtColor(result.astype(np.uint8), cv2.COLOR_RGB2GRAY)
    
    for i in range(iterations):
        # Compute gradients for edge-preserving diffusion
        if edge_preserving:
            # Compute image gradients
            grad_x = cv2.Sobel(gray, cv2.CV_32F, 1, 0, ksize=3)
            grad_y = cv2.Sobel(gray, cv2.CV_32F, 0, 1, ksize=3)
            grad_mag = np.sqrt(grad_x**2 + grad_y**2)
            
            # Edge-preserving coefficient (lower diffusion near edges)
            edge_coeff = 1.0 / (1.0 + grad_mag / 10.0)
        else:
            edge_coeff = np.ones_like(gray)
        
        # Apply diffusion to each channel
        for c in range(3):
            # Compute Laplacian
            laplacian = cv2.Laplacian(result[:, :, c], cv2.CV_32F, ksize=3)
            
            # Edge-preserving update
            update = dt * laplacian * edge_coeff
            result[:, :, c] = result[:, :, c] + update * mask_bool
        
        # Keep known regions fixed
        result = result * known_mask[..., np.newaxis] + img * known_mask[..., np.newaxis]
        result = np.clip(result, 0, 255)
        
        # Update grayscale for next iteration
        gray = cv2.cvtColor(result.astype(np.uint8), cv2.COLOR_RGB2GRAY)
    
    return result.astype(np.uint8)

print("PDE inpainting functions defined!")


## 2. Test on Sample Image


In [ ]:
# Load sample data
DAMAGED_DIR = Path('../data/damaged')
MASKS_DIR = Path('../data/masks')
GROUND_TRUTH_DIR = Path('../data/ground_truth')
RESULTS_DIR = Path('../methods/PDE/results')

ensure_dir(RESULTS_DIR)

# Get sample files
damaged_files = get_image_files(DAMAGED_DIR)
if damaged_files:
    sample_damaged = load_image(damaged_files[0])
    sample_mask = cv2.imread(str(MASKS_DIR / f"{damaged_files[0].stem}.png"), cv2.IMREAD_GRAYSCALE)
    sample_gt = load_image(GROUND_TRUTH_DIR / f"{damaged_files[0].stem}.png")
    
    print(f"Testing on: {damaged_files[0].name}")
    print(f"Image shape: {sample_damaged.shape}")
    print(f"Mask shape: {sample_mask.shape}")
    
    # Test basic PDE
    print("\nRunning basic PDE inpainting (100 iterations)...")
    result_basic = pde_inpaint(sample_damaged, sample_mask, iterations=100)
    
    # Test improved PDE
    print("Running improved PDE inpainting (200 iterations)...")
    result_improved = pde_inpaint_improved(sample_damaged, sample_mask, iterations=200)
    
    # Visualize
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    
    axes[0, 0].imshow(sample_gt)
    axes[0, 0].set_title('Ground Truth')
    axes[0, 0].axis('off')
    
    axes[0, 1].imshow(sample_damaged)
    axes[0, 1].set_title('Damaged')
    axes[0, 1].axis('off')
    
    axes[0, 2].imshow(sample_mask, cmap='gray')
    axes[0, 2].set_title('Mask')
    axes[0, 2].axis('off')
    
    axes[1, 0].imshow(result_basic)
    axes[1, 0].set_title('PDE Basic (100 iter)')
    axes[1, 0].axis('off')
    
    axes[1, 1].imshow(result_improved)
    axes[1, 1].set_title('PDE Improved (200 iter)')
    axes[1, 1].axis('off')
    
    # Difference
    diff = np.abs(result_improved.astype(float) - sample_gt.astype(float))
    axes[1, 2].imshow(diff.astype(np.uint8))
    axes[1, 2].set_title('Difference from GT')
    axes[1, 2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print("\nSample test complete!")


In [ ]:
# Process all images
print(f"Processing {len(damaged_files)} images with PDE inpainting...")

# Parameters
ITERATIONS = 200
DT = 0.1

for img_path in tqdm(damaged_files, desc="PDE Inpainting"):
    # Load damaged image and mask
    damaged_img = load_image(img_path)
    mask = cv2.imread(str(MASKS_DIR / f"{img_path.stem}.png"), cv2.IMREAD_GRAYSCALE)
    
    # Apply PDE inpainting
    restored = pde_inpaint_improved(damaged_img, mask, iterations=ITERATIONS, dt=DT)
    
    # Save result
    save_image(restored, RESULTS_DIR / f"{img_path.stem}.png")

print(f"\n✓ All {len(damaged_files)} images processed and saved to {RESULTS_DIR}")


## 4. Summary

PDE-based restoration complete! All restored images saved to `methods/PDE/results/`.

**Parameters used:**
- Iterations: 200
- Time step (dt): 0.1
- Method: Edge-preserving PDE inpainting

The PDE method works well for small to medium-sized damaged regions and preserves smooth textures.
